# Building a Neural Network Classifier in PyTorch

This tutorial walks you through building a basic neural network using **PyTorch's `nn.Module`**.  
We use the **Breast Cancer Wisconsin dataset** to perform binary classification.


### 1. Import Required Libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


### 2. Load and Preprocess the Dataset

We load the dataset, standardize the features, split into training and testing sets, and convert the data to PyTorch tensors.


In [2]:
# Load the dataset
data = load_breast_cancer()

In [3]:
# Create a DataFrame from the data
df = pd.DataFrame(data=data.data, columns=data.feature_names)

# Add the target column
df['target'] = data.target

# Optional: Display the first few rows
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [4]:
df.shape

(569, 31)

In [5]:
# Separate features and target
X = df.drop('target', axis=1)
y = df['target']

# Split the dataset: 80% train, 20% test (you can adjust the test_size as needed)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Optional: Display the shapes of the splits
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(455, 30) (114, 30) (455,) (114,)


In [6]:
print(type(X_train), type(X_test), type(y_train), type(y_test))

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'>


In [7]:
# Standardize features (mean=0, std=1)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [8]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.to_numpy(), dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.to_numpy(), dtype=torch.float32)

In [9]:
print(type(X_train_tensor), type(X_test_tensor), type(y_train_tensor), type(y_test_tensor))

<class 'torch.Tensor'> <class 'torch.Tensor'> <class 'torch.Tensor'> <class 'torch.Tensor'>


In [10]:
print(X_train_tensor.dtype, X_test_tensor.dtype, y_train_tensor.dtype, y_test_tensor.dtype)

torch.float32 torch.float32 torch.float32 torch.float32


### 3. Define the Model Class

We define a neural network using PyTorch's `nn.Module` with the following architecture:
- Linear layer with 3 hidden units
- ReLU activation
- Output layer with sigmoid activation for binary classification


In [11]:
class MySimpleNN(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(num_features, 3),
            nn.ReLU(),
            nn.Linear(3, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.network(x)

### 4. Set Training Parameters
We define the hyperparameters and loss function.

In [13]:
# define learning_rate
learning_rate = 0.1
# define epochs
epochs = 25
# define loss function
loss_function = nn.BCELoss()

### 5. Train the Model
We train the model using the SGD optimizer and track the loss at each epoch.

In [14]:
# create model
model = MySimpleNN(X_train_tensor.shape[1])

# define optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# define loop
for epoch in range(epochs):

  # forward pass
  y_pred = model(X_train_tensor)

  # loss calculate
  loss = loss_function(y_pred, y_train_tensor.view(-1,1))

  # clear gradients
  optimizer.zero_grad()

  # backward pass
  loss.backward()

  # parameters update
  optimizer.step()

  # print loss in each epoch
  print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

Epoch: 1, Loss: 0.7402001023292542
Epoch: 2, Loss: 0.6980209350585938
Epoch: 3, Loss: 0.6589018106460571
Epoch: 4, Loss: 0.6234217882156372
Epoch: 5, Loss: 0.5903750658035278
Epoch: 6, Loss: 0.5599197149276733
Epoch: 7, Loss: 0.5322003364562988
Epoch: 8, Loss: 0.5072342157363892
Epoch: 9, Loss: 0.484884113073349
Epoch: 10, Loss: 0.46498972177505493
Epoch: 11, Loss: 0.4472200572490692
Epoch: 12, Loss: 0.4313748776912689
Epoch: 13, Loss: 0.4171529710292816
Epoch: 14, Loss: 0.4044259786605835
Epoch: 15, Loss: 0.3928118646144867
Epoch: 16, Loss: 0.38214054703712463
Epoch: 17, Loss: 0.37234658002853394
Epoch: 18, Loss: 0.3631904423236847
Epoch: 19, Loss: 0.35459497570991516
Epoch: 20, Loss: 0.34638360142707825
Epoch: 21, Loss: 0.3384915888309479
Epoch: 22, Loss: 0.33094683289527893
Epoch: 23, Loss: 0.3236660063266754
Epoch: 24, Loss: 0.31651923060417175
Epoch: 25, Loss: 0.30962100625038147


### 6. Evaluate the Model
We test the model on unseen data and calculate accuracy.

In [15]:
# model evaluation
with torch.no_grad():
  y_pred = model.forward(X_test_tensor)
  y_pred = (y_pred > 0.5).float()
  accuracy = (y_pred == y_test_tensor).float().mean()
  print(f'Accuracy: {accuracy.item()}')


Accuracy: 0.5280086398124695


###  Conclusion
We built and trained a simple neural network using PyTorch.

- We used `nn.Sequential` for fast model definition.
- Training was done with `BCELoss` and `SGD`.
- We achieved a solid accuracy for binary classification.
